In [7]:
import numpy as np
import json
import os
import sys
import time
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torchattacks

from utils import imshow, image_folder_custom_label

from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F

# 기본 `log_dir` 은 "runs"이며, 여기서는 더 구체적으로 지정하였습니다
writer = SummaryWriter('runs/white_box_attack_image_net')

In [8]:
print("PyTorch", torch.__version__)
print("Torchvision", torchvision.__version__)
print("Torchattacks", torchattacks.__version__)
print("Numpy", np.__version__)

PyTorch 1.9.1+cu102
Torchvision 0.10.1+cu102
Torchattacks 3.1.0
Numpy 1.21.2


In [9]:
USE_CUDA = torch.cuda.is_available() 
print(USE_CUDA)

# 'cuda:0' if USE_CUDA else 
device = torch.device('cpu') 
print('학습을 진행하는 기기:',device)

True
학습을 진행하는 기기: cpu


In [10]:
# https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json
class_idx = json.load(open("./data/imagenet_class_index.json"))
idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]

transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(), # ToTensor : [0, 255] -> [0, 1]
    
# Using normalization for Inception v3.
# https://discuss.pytorch.org/t/how-to-preprocess-input-for-pre-trained-networks/683
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],                     
#                          std=[0.229, 0.224, 0.225])
    
# However, DO NOT USE normalization transforms here.
# Torchattacks only supports images with a range between 0 and 1.
# Thus, please refer to the model construction section.
    
])

imagnet_data = image_folder_custom_label(root='./data/imagenet', transform=transform, idx2label=idx2label)
data_loader = torch.utils.data.DataLoader(imagnet_data, batch_size=1, shuffle=False)

# 임의의 학습 이미지를 가져옵니다
dataiter = iter(data_loader)
images, labels = dataiter.next()

In [11]:
class MyModel(nn.Module):
    def __init__(self, mean, std):
        super(MyModel, self).__init__()
        self.register_buffer('mean', torch.Tensor(mean))
        self.register_buffer('std', torch.Tensor(std))
        
    def forward(self, input):
        # Broadcasting
        mean = self.mean.reshape(1, 3, 1, 1)
        std = self.std.reshape(1, 3, 1, 1)
        return (input - mean) / std
        

# model = model.eval()
myModel = MyModel(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]).eval()

In [12]:
writer.add_graph(myModel, images)
writer.close()